In [1]:
import sys
sys.path.append("..")
sys.path.append("../lmmnn/")

from utils.training_functions import *
from utils.evaluation import *
from data import dataset_preprocessing

import pickle

RS = 42

C:\Users\AndrejTschalzev\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### Download and save data from Pargent et al. by running "data/download_pargent2022_datasets.py before running this notebook

In [2]:
dataset_name = "hussain"
mode="train_val_test"
RS=42
hct=10
test_ratio=0.2
val_ratio=0.1
folds=None
target = "continuous"

data_path = f"{mode}_RS{RS}_hct{hct}"
if mode == "cv":
    data_path += f"_{folds}folds"
elif mode == "train_test":
    data_path += f"_split{1-test_ratio*100}-{test_ratio*100}"
elif mode == "train_val_test":
    data_path += f"_split{round(100-(test_ratio+val_ratio)*100)}-{round(test_ratio*100)}-{round(val_ratio*100)}"


# If no data_dict for the configuration exists, run preprocessing, else load data_dict
if not os.path.exists(f"../data/prepared/{dataset_name}/"+data_path+"/data_dict.pickle"):
    dataset_preprocessing.process_dataset(dataset_name, target, mode, RS, hct, test_ratio, val_ratio, folds)
with open(f"../data/prepared/{dataset_name}/{data_path}/data_dict.pickle", 'rb') as handle:
        data_dict = pickle.load(handle)


In [3]:
z_ohe_encoded_train = data_dict["z_ohe_encoded_train"] 
z_ohe_encoded_val = data_dict["z_ohe_encoded_val"] 
z_ohe_encoded_test = data_dict["z_ohe_encoded_test"] 

z_target_encoded_train = data_dict["z_target_encoded_train"] 
z_target_encoded_val = data_dict["z_target_encoded_val"] 
z_target_encoded_test = data_dict["z_target_encoded_test"] 

X_train = data_dict["X_train"]
Z_train = data_dict["Z_train"]
y_train = data_dict["y_train"]

X_val = data_dict["X_val"]
Z_val = data_dict["Z_val"]
y_val = data_dict["y_val"]

X_test = data_dict["X_test"]
Z_test = data_dict["Z_test"]
y_test = data_dict["y_test"]

z_cols = data_dict["z_cols"]

perf_cols = [i for i in X_train.columns if ("tnp" in i or "tfp" in i or "iap" in i)]


### Correlations to target

In [4]:
pd.DataFrame(pd.concat([X_train,pd.Series(y_train,index=X_train.index,name="target")],axis=1).corr()["target"])

,target
ge,-0.053981
arr,-0.413169
ls,-0.105440
as,0.410070
ss,0.289782
...,...
fq_5,-0.217124
sh_0,-0.154293
sh_1,0.276475
sh_2,-0.051303


In [5]:
# Prev. performance features
pd.DataFrame(pd.concat([X_train[perf_cols],pd.Series(y_train,index=X_train.index,name="target")],axis=1).corr()["target"])

,target
tnp_0,0.298173
tnp_1,-0.301747
tnp_2,-0.394062
tnp_3,0.507698
iap_0,0.451054
iap_1,-0.322819
iap_2,-0.275187
iap_3,0.224444
target,1.000000


In [6]:
config = {
    # General Parameters
    "general_parameters": {
        "target": "continuous",
        "metrics": ["mse", "r2"],
        "model_name": "tabtransformer",
                             },

    # NN Parameters
    "nn_parameters": {"epochs": 200,
                      "batch_size":  100,
                      "patience": 5,
                      "stop_metric": "val_mse"},
    
    "embed_parameters":
        {"embed_dims_method": "sqrt"}
     

        }

In [7]:
train_data = [X_train, Z_train, y_train, z_ohe_encoded_train, z_target_encoded_train]
val_data = [X_val, Z_val, y_val, z_ohe_encoded_val, z_target_encoded_val]
test_data = [X_test, Z_test, y_test, z_ohe_encoded_test, z_target_encoded_test]


In [15]:
X_train_perfonly = X_train[perf_cols]
X_val_perfonly = X_val[perf_cols]
X_test_perfonly = X_test[perf_cols]

z_target_encoded_train_perfonly = z_target_encoded_train
z_target_encoded_val_perfonly = z_target_encoded_val
z_target_encoded_test_perfonly = z_target_encoded_test

z_ohe_encoded_train_perfonly = z_ohe_encoded_train
z_ohe_encoded_val_perfonly = z_ohe_encoded_val
z_ohe_encoded_test_perfonly = z_ohe_encoded_test

Z_train_perfonly = Z_train
Z_val_perfonly = Z_val
Z_test_perfonly = Z_test

train_data_perfonly = [X_train_perfonly, Z_train_perfonly, y_train, z_ohe_encoded_train_perfonly, z_target_encoded_train_perfonly]
val_data_perfonly = [X_val_perfonly, Z_val_perfonly, y_val, z_ohe_encoded_val_perfonly, z_target_encoded_val_perfonly]
test_data_perfonly = [X_test_perfonly, Z_test_perfonly, y_test, z_ohe_encoded_test_perfonly, z_target_encoded_test_perfonly]


In [9]:
X_train_noperf = X_train.drop(perf_cols,axis=1)
X_val_noperf = X_val.drop(perf_cols, axis=1)
X_test_noperf = X_test.drop(perf_cols,axis=1)

train_data_noperf = [X_train_noperf, Z_train, y_train, z_ohe_encoded_train, z_target_encoded_train]
val_data_noperf = [X_val_noperf, Z_val, y_val, z_ohe_encoded_val, z_target_encoded_val]
test_data_noperf = [X_test_noperf, Z_test, y_test, z_ohe_encoded_test, z_target_encoded_test]


## Train the models to evaluate

In [11]:
save_path = f"../results/{dataset_name}/{data_path}"
if not os.path.exists(f"{save_path}/results_dict.pickle"):
    results_dict = train_lowcard_models(train_data, val_data, test_data, config, RS=RS, save_results=False,save_path=save_path)
else:
    with open(f"{save_path}/results_dict.pickle", 'rb') as handle:
        results_dict = pickle.load(handle)

results = results_dict["results"]

Load base model
Train XGBoost without z features
Train Linear Model without z features
Train NN without Z features
Epoch 1/200
1/1 [==============================] - 1s 951ms/step - loss: 4.7630 - r2: -3.7630 - mse: 4.7630 - val_loss: 3.4654 - val_r2: -1.3890 - val_mse: 3.4654
Epoch 2/200
1/1 [==============================] - 0s 65ms/step - loss: 2.2796 - r2: -1.2796 - mse: 2.2796 - val_loss: 1.7180 - val_r2: -0.1844 - val_mse: 1.7180
Epoch 3/200
1/1 [==============================] - 0s 54ms/step - loss: 1.0510 - r2: -0.0510 - mse: 1.0510 - val_loss: 0.8637 - val_r2: 0.4046 - val_mse: 0.8637
Epoch 4/200
1/1 [==============================] - 0s 55ms/step - loss: 0.9625 - r2: 0.0375 - mse: 0.9625 - val_loss: 0.5759 - val_r2: 0.6030 - val_mse: 0.5759
Epoch 5/200
1/1 [==============================] - 0s 52ms/step - loss: 1.3413 - r2: -0.3413 - mse: 1.3413 - val_loss: 0.4728 - val_r2: 0.6741 - val_mse: 0.4728
Epoch 6/200
1/1 [==============================] - 0s 62ms/step - loss: 1.2719

In [16]:
save_path = f"../results/{dataset_name}/{data_path}"
if not os.path.exists(f"{save_path}/results_dict_perfonly.pickle"):
    results_dict_perfonly = train_lowcard_models(train_data_perfonly, val_data_perfonly, test_data_perfonly, config, RS=RS, save_results=False,save_path=save_path)
else:
    with open(f"{save_path}/results_dict_perfonly.pickle", 'rb') as handle:
        results_dict_perfonly = pickle.load(handle)

results_perfonly = results_dict_perfonly["results"]

Load base model
Train XGBoost without z features
Train Linear Model without z features
Train NN without Z features
Epoch 1/200
1/1 [==============================] - 1s 872ms/step - loss: 2.9190 - r2: -1.9190 - mse: 2.9190 - val_loss: 2.4827 - val_r2: -0.7116 - val_mse: 2.4827
Epoch 2/200
1/1 [==============================] - 0s 48ms/step - loss: 2.6738 - r2: -1.6738 - mse: 2.6738 - val_loss: 2.2742 - val_r2: -0.5679 - val_mse: 2.2742
Epoch 3/200
1/1 [==============================] - 0s 62ms/step - loss: 2.1714 - r2: -1.1714 - mse: 2.1714 - val_loss: 2.0767 - val_r2: -0.4317 - val_mse: 2.0767
Epoch 4/200
1/1 [==============================] - 0s 54ms/step - loss: 2.1163 - r2: -1.1163 - mse: 2.1163 - val_loss: 1.8877 - val_r2: -0.3014 - val_mse: 1.8877
Epoch 5/200
1/1 [==============================] - 0s 53ms/step - loss: 2.4082 - r2: -1.4082 - mse: 2.4082 - val_loss: 1.7065 - val_r2: -0.1764 - val_mse: 1.7065
Epoch 6/200
1/1 [==============================] - 0s 46ms/step - loss: 1.

In [12]:
save_path = f"../results/{dataset_name}/{data_path}"
if not os.path.exists(f"{save_path}/results_dict_noperf.pickle"):
    results_dict_noperf = train_lowcard_models(train_data_noperf, val_data_noperf, test_data_noperf, config, RS=RS, save_results=False,save_path=save_path)
else:
    with open(f"{save_path}/results_dict.pickle_noperf", 'rb') as handle:
        results_dict_noperf = pickle.load(handle)

results_noperf = results_dict_noperf["results"]

Load base model
Train XGBoost without z features
Train Linear Model without z features
Train NN without Z features
Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 4.0812 - r2: -3.0812 - mse: 4.0812 - val_loss: 1.4565 - val_r2: -0.0041 - val_mse: 1.4565
Epoch 2/200
1/1 [==============================] - 0s 46ms/step - loss: 1.7912 - r2: -0.7912 - mse: 1.7912 - val_loss: 1.0470 - val_r2: 0.2782 - val_mse: 1.0470
Epoch 3/200
1/1 [==============================] - 0s 38ms/step - loss: 1.2468 - r2: -0.2468 - mse: 1.2468 - val_loss: 0.9218 - val_r2: 0.3645 - val_mse: 0.9218
Epoch 4/200
1/1 [==============================] - 0s 41ms/step - loss: 1.0910 - r2: -0.0910 - mse: 1.0910 - val_loss: 0.8138 - val_r2: 0.4390 - val_mse: 0.8138
Epoch 5/200
1/1 [==============================] - 0s 58ms/step - loss: 1.4592 - r2: -0.4592 - mse: 1.4592 - val_loss: 0.7037 - val_r2: 0.5149 - val_mse: 0.7037
Epoch 6/200
1/1 [==============================] - 0s 48ms/step - loss: 1.1692 - 

## Evaluation

### Performance on whole data

In [18]:
models_use = ["LR", "XGB", "NN"]

In [19]:
results_df = pd.DataFrame(results).transpose().loc[models_use].sort_values("MSE Test",ascending=False).round(4)


In [20]:
results_df[["MSE Train", "R2 Train", "MSE Test", "R2 Test"]].style.highlight_min(subset=["MSE Train", "MSE Test"], color = 'lightgreen', axis = 0).highlight_max(subset=["R2 Train", "R2 Test"], color = 'lightgreen', axis = 0)

,MSE Train,R2 Train,MSE Test,R2 Test
LR,0.190700,0.809300,1347749.750000,-2069905.728000
NN,0.400000,0.600000,0.988300,-0.517800
XGB,0.000000,1.000000,0.506400,0.222300


### Modeling results only with previous performance

In [22]:
results_df_perfonly = pd.DataFrame(results_perfonly).transpose().loc[models_use].sort_values("MSE Test",ascending=False).round(4)


In [23]:
results_df_perfonly[["MSE Train", "R2 Train", "MSE Test", "R2 Test"]].style.highlight_min(subset=["MSE Train", "MSE Test"], color = 'lightgreen', axis = 0).highlight_max(subset=["R2 Train", "R2 Test"], color = 'lightgreen', axis = 0)

,MSE Train,R2 Train,MSE Test,R2 Test
NN,0.609600,0.390400,0.596100,0.084500
XGB,0.437100,0.562900,0.374200,0.425300
LR,0.462600,0.537500,0.360000,0.447200


### Modeling results without previous performance

In [24]:
results_df_noperf = pd.DataFrame(results_noperf).transpose().loc[models_use].sort_values("MSE Test",ascending=False).round(4)


In [25]:
results_df_noperf[["MSE Train", "R2 Train", "MSE Test", "R2 Test"]].style.highlight_min(subset=["MSE Train", "MSE Test"], color = 'lightgreen', axis = 0).highlight_max(subset=["R2 Train", "R2 Test"], color = 'lightgreen', axis = 0)

,MSE Train,R2 Train,MSE Test,R2 Test
LR,0.243100,0.756900,4670.192900,-7171.595200
NN,0.586800,0.413200,0.918400,-0.410500
XGB,0.000000,1.000000,0.511600,0.214300
